# Stockage des matrices

## Stockage plein

Naturellement, les tableaux à deux dimensions sont bien adaptés pour stocker les matrices. Par exemple, on peut utiliser un tableau `numpy` bidimensionnel pour stocker la matrice :

$$
A = \begin{bmatrix}
1 & 0 & 0 & 3 \\
2 & 5 & 0 & 7 \\
0 & 4 & 2 & 0 \\
5 & 0 & 0 & 1
\end{bmatrix}
$$

In [1]:
import numpy as np
A = np.array([[1,0,0,3], [2,5,0,7], [0,4,2,0], [5,0,0,1]])
print(A)

[[1 0 0 3]
 [2 5 0 7]
 [0 4 2 0]
 [5 0 0 1]]


Une matrice carré de taille $(n × n)$ dont les coefficients sont stockés sur les flottants 64 bits occupe en mémoire $8 × (n × n)$ octets. 

Exemples :
* une matrice de taille $(1024 \times 1024)$ occupe 8 Mio
* une matrice de taille $(2048 \times 2048)$ occupe 32 Mio
* une matrice de taille $(65536 \times 65536)$ occupe 32 Gio
* une matrice de taille $(1048576 \times 1048576)$ occupe 8 Tio

La fonction `nbytes` retourne l'occupation mémoire en octets 

In [7]:
n = 1024
Rand = np.random.rand(n,n)
print(f"Taille de la matrice de taille ({n}x{n}) : {Rand.nbytes/1024} Mio")

Taille de la matrice de taille (1024x1024) : 8192.0 Mio


Les matrices issues de la discrétisation spatiale d'équations aux dérivées partielles contiennent majoritairement des zéros.

Pour stocker ces matrices, on n'utilise pas des tableaux à deux dimensions mais des structures de données adaptées au caractère creux de ces matrices.

## Matrice creuse : COO (Coordinate Format)

La structure de données COO peut être représentée par :

* un tableau `data` contenant les coefficients non nuls de la matrice (dans n'importe quel ordre)
* un tableau `row` contenant les indices de ligne de chaque élément de `data`
* un tableau `col` contenant les indices de colonne de chaque élément de `data`

Exemple pour la matrice :

$$
A = \begin{bmatrix}
1 & 0 & 0 & 3 \\
2 & 5 & 0 & 7 \\
0 & 4 & 2 & 0 \\
5 & 0 & 0 & 1
\end{bmatrix}
$$

In [8]:
from scipy.sparse import coo_matrix

data = np.array([1, 3, 2, 5, 4, 5, 1, 2, 7])

row = np.array([0, 0, 1, 1, 2, 3, 3, 2, 1])
col = np.array([0, 3, 0, 1, 1, 0, 3, 2, 3])

A_coo = coo_matrix((data, (row, col)), shape=(4, 4))
A_coo.todense()

matrix([[1, 0, 0, 3],
        [2, 5, 0, 7],
        [0, 4, 2, 0],
        [5, 0, 0, 1]])

## Matrice creuse : CSR (Compressed Sparse Row)

La structure de données CSR peut être représentée par :

* un tableau `data` contenant les coefficients non nuls de la matrice rangés ligne par ligne
* un tableau `col` contenant les indices de colonne de chaque élément de `data`
* un tableau `row_ptr` dont l'élément `i`  contient l'index dans les tableaux `data` et `col` de la première entrée non nulle de la ligne `i` de la matrice


Exemple pour la matrice :

$$
A = \begin{bmatrix}
1 & 0 & 0 & 3 \\
2 & 5 & 0 & 7 \\
0 & 4 & 2 & 0 \\
5 & 0 & 0 & 1
\end{bmatrix}
$$

In [ ]:
from scipy.sparse import csr_matrix

data = np.array([1, 3, 2, 5, 7, 4, 2, 5, 1])

col = [0, 3, 0, 1, 2, 1, 2, 0, 3]
row_ptr = [0, 2, 5, 7, 9]

A_csr = csr_matrix((data, col, row_ptr), shape=(4, 4))
A_csr.todense()

## Matrice creuse : CSC (Compressed Sparse Column)

La structure de données CSC peut être représentée par :

* un tableau `data` contenant les coefficients non nuls de la matrice rangés colonne par colonne
* un tableau `row` contenant les indices de ligne de chaque élément de `data`
* un tableau `col_ptr` dont l'élément `i`  contient l'index dans les tableaux `data` et `row` de la première entrée non nulle de la colonne `i` de la matrice

Exemple :

$$
A = \begin{bmatrix}
1 & 0 & 0 & 3 \\
2 & 5 & 0 & 7 \\
0 & 4 & 2 & 0 \\
5 & 0 & 0 & 1
\end{bmatrix}
$$

In [ ]:
from scipy.sparse import csr_matrix

data = np.array([1, 3, 2, 5, 7, 4, 2, 5, 1])

row = [0, 1, 3, 1, 2, 1, 2, 0, 3]
col_ptr = [0, 3, 5, 6, 9]

A_csr = csr_matrix((data, col, row_ptr), shape=(4, 4))
A_csr.todense()

## Matrice creuse : rangement diagonal

La structure de données à rangement diagonal peut être représentée par :

* un tableau `diags` à deux dimensions ab contenant les coefficients de chaque diagonale de la matrice
* un tableau d'entier `offset` contenant la postion de chaque diagonale relativement à la diagonale principale

Remarques : cette structure de données peut stocker des éléments non nuls.

Exemple :

$$
A = \begin{bmatrix}
5 & 2 & 6  & 0  & 0  \\
1 & 4 & 2  & 5  & 0  \\
0 & 1 & 3  & 2  & 1  \\
0 & 0 & 1  & 2  & 0  \\
0 & 0 & 0  & 1  & 1
\end{bmatrix}
$$

In [ ]:
from scipy.sparse import dia_matrix

data = np.array([[5, 4, 3, 2, 1], [1, 1, 1, 1, 0], [2, 2, 2, 0, 0], [6, 5, 1, 0, 0]])

offset = np.array([0, -1, 1, 2])

A_dia = dia_matrix((data, offset), shape=(5, 5))
A_dia.todense()

## Références 

* [Sparse Matrix Storage Formats, *Jack Dongarra*](http://www.netlib.org/utk/people/JackDongarra/etemplates/node372.html)
* [Classes des matrice creuses de scipy](https://docs.scipy.org/doc/scipy/reference/sparse.html)